**Data retrieved from** https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses

**Task:** Create a heatmap for aircrash instances in the united states, imputing missing coordinates using Google Geocoding API

In [76]:
# Initial reading of df
import pandas as pd
import plotly.express as px # Data visualisation library

airdf = pd.read_csv("AviationData.csv", encoding="ISO-8859-1")
airdf

C:\Users\fluff\AppData\Local\Temp\ipykernel_5116\651279179.py:5: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  airdf = pd.read_csv("AviationData.csv", encoding="ISO-8859-1")


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [77]:
#Drop all countries outside of the US
airdf = airdf[airdf['Country'] == 'United States']

In [78]:
#Drop categories not relevant to task
airdf.drop(['Investigation.Type', 'Airport.Code', 'Aircraft.damage', 'Publication.Date', 'Schedule', 'FAR.Description', 
            'Event.Date', 'Air.carrier', 'Report.Status', 'Total.Serious.Injuries', 'Airport.Name', 'Injury.Severity', 'Aircraft.Category',
            'Total.Minor.Injuries', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'Purpose.of.flight', 'Model', 'Country', 'Event.Id', 
            'Accident.Number', 'Registration.Number', 'Total.Fatal.Injuries', 'Total.Uninjured', 'Make', 'Weather.Condition'], axis=1, inplace=True)

airdf

C:\Users\fluff\AppData\Local\Temp\ipykernel_5116\118574494.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airdf.drop(['Investigation.Type', 'Airport.Code', 'Aircraft.damage', 'Publication.Date', 'Schedule', 'FAR.Description',


,Location,Latitude,Longitude,Broad.phase.of.flight
0,"MOOSE CREEK, ID",NaN,NaN,Cruise
1,"BRIDGEPORT, CA",NaN,NaN,Unknown
2,"Saltville, VA",36.922223,-81.878056,Cruise
3,"EUREKA, CA",NaN,NaN,Cruise
4,"Canton, OH",NaN,NaN,Approach
...,...,...,...,...
88884,"Annapolis, MD",NaN,NaN,NaN
88885,"Hampton, NH",NaN,NaN,NaN
88886,"Payson, AZ",341525N,1112021W,NaN
88887,"Morgan, UT",NaN,NaN,NaN


In [79]:
airdf.isna().sum()

Location                    11
Latitude                 49983
Longitude                49993
Broad.phase.of.flight    21090
dtype: int64

In [80]:
# Check the dataframe data types
airdf.dtypes

Location                 object
Latitude                 object
Longitude                object
Broad.phase.of.flight    object
dtype: object